In this notebook, we will run and verify the SWOT Assimilated DiScharge (SADS) algorithm against data from the SWOT Pepsi-1 challenge. We start by adding the SADS modules to the load path and importing them. 

In [17]:
@everywhere push!(LOAD_PATH,"../src")
using CrossSections
using Gvf
using Kalman
using NCDatasets
using Distributions

We load the data from the Po dataset file

In [12]:
ds = Dataset("Po.nc");
xs = NCDatasets.group(ds, "XS_Timeseries");
ri = NCDatasets.group(ds, "River_Info");

In [14]:
qwbm = ri["QWBM"][1];
x = xs["X"][:];
W = xs["W"][:, 1];
H = xs["H"][:, 1];
Q = xs["Q"][:, 1];

We then generate an ensemble of discharge, bed slopes, roughness coefficients

In [19]:
nens = 50;
Qe = rand(Normal(qwbm, 200), nens);
ne = rand(Uniform(0.01, 0.05), nens);
S0 = rand(Normal(0.00019, 0.00005), nens);

and then we can generate an ensemble of depth profiles using the GVF model after we calculate a downstream depth as the boundary condition. We use an assumption of normal flow to estimate that depth along with the observed water surface slope

In [ ]:
he = zeros(15, nens);
for i in 1:nens
    he[:, i] = gvf(Qe[i], mean(ye), S0[i], mean(n), x, w);
end